In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import csv


In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/tag-genome/tag-genome'

movies_fp        = f"{base_path}/movies.dat"
tags_fp          = f"{base_path}/tags.dat"
tag_relevance_fp = f"{base_path}/tag_relevance.dat"

movies = pd.read_csv(
    movies_fp,
    sep='\t',
    header=None,
    names=['movieId', 'title', 'genres'],
    engine='python',
    quoting=csv.QUOTE_NONE
)

tags = pd.read_csv(
    tags_fp,
    sep='\t',
    header=None,
    names=['userId', 'movieId', 'tag', 'timestamp'],
    engine='python',
    quoting=csv.QUOTE_NONE
)

tag_rel = pd.read_csv(
    tag_relevance_fp,
    sep='\t',
    header=None,
    names=['movieId', 'tagId', 'relevance'],
    engine='python',
    quoting=csv.QUOTE_NONE
)


In [ ]:
tags_df = pd.read_csv(
    tags_fp,
    sep='\t',
    header=None,
    names=["tagId", "tag"],
    engine="python",
    quoting=csv.QUOTE_NONE
)

tags_clean = tags_df[['tagId', 'tag']].drop_duplicates().reset_index(drop=True)
tags_clean = tags_clean.rename(columns={"tagId": "tag", "tag": "tagId"})
tags_clean = tags_clean[["tagId", "tag"]]
tags_clean['tagId'] = tags_clean['tagId'].astype(int)
tags_clean['tag'] = tags_clean['tag'].astype(str)

tagId_to_text = dict(zip(tags_clean['tagId'], tags_clean['tag']))


In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

tag_texts = list(tagId_to_text.values())
tag_embeddings = embedder.encode(tag_texts, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def recommend_movies_from_mood(user_prompt, top_k_tags=5, top_n_movies=10, tag_rel=tag_rel, movies=movies):
    user_embedding = embedder.encode(user_prompt, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(user_embedding, tag_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k_tags)

    top_tag_indices = top_results.indices.cpu().numpy()
    top_tags = [tag_texts[idx] for idx in top_tag_indices]
    print(f"\nTop matching tags for your mood: {top_tags}\n")

    tag_text_to_id = {v: k for k, v in tagId_to_text.items()}
    top_tag_ids = [tag_text_to_id[tag] for tag in top_tags]

    matched_movies = tag_rel[tag_rel['tagId'].isin(top_tag_ids)].copy()
    movie_scores = matched_movies.groupby('movieId')['relevance'].sum()
    top_movie_ids = movie_scores.sort_values(ascending=False).head(top_n_movies).index.tolist()

    recommended_movies = movies[movies['movieId'].isin(top_movie_ids)][['movieId', 'title']]
    recommended_movies = recommended_movies.merge(movie_scores, left_on='movieId', right_index=True)
    recommended_movies = recommended_movies.sort_values(by='relevance', ascending=False)

    return recommended_movies


In [ ]:
user_mood = "I'm feeling adventurous and want an emotional story."
recommendations = recommend_movies_from_mood(user_mood)
print(recommendations)

user_mood = "I want a dark psychological thriller."
recommendations = recommend_movies_from_mood(user_mood)
print(recommendations)

user_mood = "I want a light-hearted, feel-good comedy."
recommendations = recommend_movies_from_mood(user_mood)
print(recommendations)



Top matching tags for your mood: ['love story', 'storytelling', 'romantic', 'adventure', 'suspense']

      movieId                                          title  relevance
7266    31429                      Aliens of the Deep (2005)      2.376
9683   103659  Justice League: The Flashpoint Paradox (2013)      2.342
6503     8361                 Day After Tomorrow, The (2004)      2.203
2271     2571                             Matrix, The (1999)      2.095
8877    72998                                  Avatar (2009)      2.086
8848    72378                                    2012 (2009)      2.084
9150    82041                         Loved Ones, The (2009)      2.074
9043    79132                               Inception (2010)      2.070
5724     6774                              Videodrome (1983)      2.063
1616     1876                             Deep Impact (1998)      2.055

Top matching tags for your mood: ['thriller', 'dark comedy', 'supernatural', 'film noir', 'dark']

     